In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

import sys
sys.path.append('..')

from tracks_interactions.db.db_model import CellDB, TrackDB,Base
from tracks_interactions.db.db_functions import get_descendants

In [8]:
# define pathways

org_db_path = r'D:\kasia\tracking\E6_exp\code\gardener_20_dev\cells_database_2tables.db'

new_db_path = r'../tests/fixtures/db_2tables_test.db'

In [9]:
engine = create_engine(f'sqlite:///{org_db_path}')
session = sessionmaker(bind=engine)()

new_engine = create_engine(f'sqlite:///{new_db_path}')
Base.metadata.create_all(new_engine)
new_session = sessionmaker(bind=new_engine)()

In [12]:
# get all the descendants of the selected roots
# don't ask for not roots as their parameters will be incorrect in the test database
selected_roots = [15854,17173]
selected_track_id_list = []

for root in selected_roots:
    selected_track_id_list.extend([x[0] for x in get_descendants(session,root)])

selected_track_id_list

[15854,
 15855,
 15856,
 17173,
 17175,
 17174,
 17176,
 17177,
 17178,
 17179,
 17180,
 17181]

In [13]:
# copy the selected tracks and cells to the new database

for selected_track_id in selected_track_id_list:
    
    cells = session.query(CellDB).filter(CellDB.track_id == selected_track_id).all()
    tracks = session.query(TrackDB).filter(TrackDB.track_id == selected_track_id).all()

    for cell in cells:
        new_session.add(cell)

    for track in tracks:
        new_session.add(track)

    new_session.commit()

InvalidRequestError: Object '<CellDB at 0x2e1710719f0>' is already attached to session '1' (this is '2')

In [ ]:
# clean up
session.close()
new_session.close()